In [ ]:
<h4>Tensorflow Notes</h4>
<p>Most tensorflow examples use the default graph. This is not a good practice in general</p>
<p></p>
To define a graph use: 
g = tf.Graph()
<p></p>
TF Jupyter notebook is single threaded with a placeholder for the 
input data taken from sklearn. This is not good practice in general
<p></p>


In [34]:
import tensorflow as tf
import numpy as np


# to reshape (5,1) back to (5,) you can use ...
#shape tests
#
X_XOR=np.array([[0,0],[0,1],[1,0],[1,1]])
Y_XOR = np.array([0,1,1,0])

tf.reset_default_graph()
#why do the above? get memory leak, keeps on adding variables and allocating space. 
#unstack:1 will get incremented to unstack:2, etc...everytime you run this you get new allocations
#user program can cause OOM. 

print("np shape:",X_XOR.shape)

X1a = tf.placeholder(tf.float32, shape=([None,len(X_XOR)]), name="X1a")
X2a = tf.constant(X_XOR,name="X2a")
normal_dist = tf.truncated_normal(shape=[10,1],mean=0.0, stddev=.01,dtype=tf.float32, name='normaldist')
weight = tf.Variable(normal_dist, name='weight')
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(X1a)
    print(X2a)

X1a_static_shape = X1a.shape.as_list()
X1a_dynamic_shape = tf.unstack(tf.shape(X1a))

X2a_static_shape = X1a.shape.as_list()
X2a_dynamic_shape = tf.unstack(tf.shape(X1a))

weight_static_shape = weight.shape.as_list()
weight_dynamic_shape = tf.unstack(tf.shape(weight))

print("X1a_static_shape:",X1a_static_shape, "X1a_dynamic_shape:",X1a_dynamic_shape)
print("X2a_static_shape:",X2a_static_shape, "X2a_dynamic_shape:",X2a_dynamic_shape)

print("weight:",weight_static_shape, "weight_dynamic_shape:",weight_dynamic_shape)


np shape: (4, 2)
Tensor("X1a:0", shape=(?, 4), dtype=float32)
Tensor("X2a:0", shape=(4, 2), dtype=int64)
X1a_static_shape: [None, 4] X1a_dynamic_shape: [<tf.Tensor 'unstack:0' shape=() dtype=int32>, <tf.Tensor 'unstack:1' shape=() dtype=int32>]
X2a_static_shape: [None, 4] X2a_dynamic_shape: [<tf.Tensor 'unstack_1:0' shape=() dtype=int32>, <tf.Tensor 'unstack_1:1' shape=() dtype=int32>]
weight: [10, 1] weight_dynamic_shape: [<tf.Tensor 'unstack_2:0' shape=() dtype=int32>, <tf.Tensor 'unstack_2:1' shape=() dtype=int32>]


import tensorflow as tf


var0 = tf.Variable([1.0, 2.0], dtype=tf.float32)
var1 = tf.Variable([3.0, 4.0], dtype=tf.float32)
grads0 = tf.constant([0.1, 0.1], dtype=tf.float32)
grads1 = tf.constant([0.01, 0.01], dtype=tf.float32)
with tf.Session() as sess:
    sgd_op = tf.train.GradientDescentOptimizer(3.0).apply_gradients(
        zip([grads0, grads1], [var0, var1]))
    tf.global_variables_initializer().run()
    # Fetch params to validate initial values
    #self.assertAllCloseAccordingToType([3.0, 4.0], var1.eval())
    # Run 1 step of sgd
    sgd_op.run()
    print("var0:",sess.run(var0))
    print("var1:",sess.run(var1))
    print("grads0:",sess.run(grads0))
    print("grads1:",sess.run(grads1))
    
#this is funny. The learning rate is 3.0 so 3.0*0.1 is .3. But when you do the multiply
#you dont get .7. Is a little off bc rounding error. 
#


In [61]:
# updating a variable; can't update a tensor b/c it is immutable. 
#a scatter updates only the first dimension in an array. 

# this a better style, define the graph first
#
g = tf.Graph()
with g.as_default():
    a = tf.Variable(initial_value=[[0, 0, 0, 0],[0, 0, 0, 0]])
    b = tf.scatter_update(a, [0, 1], [[1, 0, 0, 0], [1, 0, 0, 0]])

with tf.Session(graph=g) as sess:
    sess.run(tf.initialize_all_variables())
    print (sess.run(a))
    print (sess.run(b))

#scatter used to distribute cells to larger/smaller matrices
#this for distributed copy


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[0 0 0 0]
 [0 0 0 0]]
[[1 0 0 0]
 [1 0 0 0]]


In [18]:
#tf practice; from the Udacity website
# do not do this
# 1) this is very very very old. The current tf documentation is light years ahead of this
#    https://www.tensorflow.org/get_started/mnist/pros
# 2) this isn't good programming practice. Use tesorflow estimator as an example
# 3) code is incorrect


#copied verbatim. 
#learning rate

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True,reshape=False)


learning_rate = .001
training_epochs = 20
batch_size = 20
display_step = 1
n_input = 784
n_classes = 10
n_hidden_layer = 256

#Weights and biases
weights = {
    'hidden_layer' : tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer,n_classes]))    
}

biases = {
    'hidden_layer' : tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


#Input
x = tf.placeholder("float", [None,28,28,1]) #change to tf.float32
y = tf.placeholder("float", [None,n_classes]) 

x_flat = tf.reshape(x,[-1,n_input]) #this is different from numpy reshape, -1 vs. none reshape

layer_1 = tf.add(tf.matmul(x_flat,weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

#optimizer

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        print('total batch,',total_batch)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer,feed_dict={x:batch_x, y:batch_y})
        print('epoch',epoch)
    print('done')     

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
total batch, 2750
i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99
i: 100
i: 101
i: 102
i: 103
i: 104
i: 105
i: 106
i: 107
i: 108
i: 109
i: 110
i: 111
i: 112
i: 113
i: 114
i: 115
i: 116
i: 117
i: 118
i: 119
i: 120
i: 121
i: 122
i: 123
i: 124
i: 125
i: 126
i: 127
i: 128
i: 129
i: 130
i: 131
i: 132
i: 

i: 1171
i: 1172
i: 1173
i: 1174
i: 1175
i: 1176
i: 1177
i: 1178
i: 1179
i: 1180
i: 1181
i: 1182
i: 1183
i: 1184
i: 1185
i: 1186
i: 1187
i: 1188
i: 1189
i: 1190
i: 1191
i: 1192
i: 1193
i: 1194
i: 1195
i: 1196
i: 1197
i: 1198
i: 1199
i: 1200
i: 1201
i: 1202
i: 1203
i: 1204
i: 1205
i: 1206
i: 1207
i: 1208
i: 1209
i: 1210
i: 1211
i: 1212
i: 1213
i: 1214
i: 1215
i: 1216
i: 1217
i: 1218
i: 1219
i: 1220
i: 1221
i: 1222
i: 1223
i: 1224
i: 1225
i: 1226
i: 1227
i: 1228
i: 1229
i: 1230
i: 1231
i: 1232
i: 1233
i: 1234
i: 1235
i: 1236
i: 1237
i: 1238
i: 1239
i: 1240
i: 1241
i: 1242
i: 1243
i: 1244
i: 1245
i: 1246
i: 1247
i: 1248
i: 1249
i: 1250
i: 1251
i: 1252
i: 1253
i: 1254
i: 1255
i: 1256
i: 1257
i: 1258
i: 1259
i: 1260
i: 1261
i: 1262
i: 1263
i: 1264
i: 1265
i: 1266
i: 1267
i: 1268
i: 1269
i: 1270
i: 1271
i: 1272
i: 1273
i: 1274
i: 1275
i: 1276
i: 1277
i: 1278
i: 1279
i: 1280
i: 1281
i: 1282
i: 1283
i: 1284
i: 1285
i: 1286
i: 1287
i: 1288
i: 1289
i: 1290
i: 1291
i: 1292
i: 1293
i: 1294
i: 1295


i: 2236
i: 2237
i: 2238
i: 2239
i: 2240
i: 2241
i: 2242
i: 2243
i: 2244
i: 2245
i: 2246
i: 2247
i: 2248
i: 2249
i: 2250
i: 2251
i: 2252
i: 2253
i: 2254
i: 2255
i: 2256
i: 2257
i: 2258
i: 2259
i: 2260
i: 2261
i: 2262
i: 2263
i: 2264
i: 2265
i: 2266
i: 2267
i: 2268
i: 2269
i: 2270
i: 2271
i: 2272
i: 2273
i: 2274
i: 2275
i: 2276
i: 2277
i: 2278
i: 2279
i: 2280
i: 2281
i: 2282
i: 2283
i: 2284
i: 2285
i: 2286
i: 2287
i: 2288
i: 2289
i: 2290
i: 2291
i: 2292
i: 2293
i: 2294
i: 2295
i: 2296
i: 2297
i: 2298
i: 2299
i: 2300
i: 2301
i: 2302
i: 2303
i: 2304
i: 2305
i: 2306
i: 2307
i: 2308
i: 2309
i: 2310
i: 2311
i: 2312
i: 2313
i: 2314
i: 2315
i: 2316
i: 2317
i: 2318
i: 2319
i: 2320
i: 2321
i: 2322
i: 2323
i: 2324
i: 2325
i: 2326
i: 2327
i: 2328
i: 2329
i: 2330
i: 2331
i: 2332
i: 2333
i: 2334
i: 2335
i: 2336
i: 2337
i: 2338
i: 2339
i: 2340
i: 2341
i: 2342
i: 2343
i: 2344
i: 2345
i: 2346
i: 2347
i: 2348
i: 2349
i: 2350
i: 2351
i: 2352
i: 2353
i: 2354
i: 2355
i: 2356
i: 2357
i: 2358
i: 2359
i: 2360


i: 678
i: 679
i: 680
i: 681
i: 682
i: 683
i: 684
i: 685
i: 686
i: 687
i: 688
i: 689
i: 690
i: 691
i: 692
i: 693
i: 694
i: 695
i: 696
i: 697
i: 698
i: 699
i: 700
i: 701
i: 702
i: 703
i: 704
i: 705
i: 706
i: 707
i: 708
i: 709
i: 710
i: 711
i: 712
i: 713
i: 714
i: 715
i: 716
i: 717
i: 718
i: 719
i: 720
i: 721
i: 722
i: 723
i: 724
i: 725
i: 726
i: 727
i: 728
i: 729
i: 730
i: 731
i: 732
i: 733
i: 734
i: 735
i: 736
i: 737
i: 738
i: 739
i: 740
i: 741
i: 742
i: 743
i: 744
i: 745
i: 746
i: 747
i: 748
i: 749
i: 750
i: 751
i: 752
i: 753
i: 754
i: 755
i: 756
i: 757
i: 758
i: 759
i: 760
i: 761
i: 762
i: 763
i: 764
i: 765
i: 766
i: 767
i: 768
i: 769
i: 770
i: 771
i: 772
i: 773
i: 774
i: 775
i: 776
i: 777
i: 778
i: 779
i: 780
i: 781
i: 782
i: 783
i: 784
i: 785
i: 786
i: 787
i: 788
i: 789
i: 790
i: 791
i: 792
i: 793
i: 794
i: 795
i: 796
i: 797
i: 798
i: 799
i: 800
i: 801
i: 802
i: 803
i: 804
i: 805
i: 806
i: 807
i: 808
i: 809
i: 810
i: 811
i: 812
i: 813
i: 814
i: 815
i: 816
i: 817
i: 818
i: 819
i: 820

i: 1839
i: 1840
i: 1841
i: 1842
i: 1843
i: 1844
i: 1845
i: 1846
i: 1847
i: 1848
i: 1849
i: 1850
i: 1851
i: 1852
i: 1853
i: 1854
i: 1855
i: 1856
i: 1857
i: 1858
i: 1859
i: 1860
i: 1861
i: 1862
i: 1863
i: 1864
i: 1865
i: 1866
i: 1867
i: 1868
i: 1869
i: 1870
i: 1871
i: 1872
i: 1873
i: 1874
i: 1875
i: 1876
i: 1877
i: 1878
i: 1879
i: 1880
i: 1881
i: 1882
i: 1883
i: 1884
i: 1885
i: 1886
i: 1887
i: 1888
i: 1889
i: 1890
i: 1891
i: 1892
i: 1893
i: 1894
i: 1895
i: 1896
i: 1897
i: 1898
i: 1899
i: 1900
i: 1901
i: 1902
i: 1903
i: 1904
i: 1905
i: 1906
i: 1907
i: 1908
i: 1909
i: 1910
i: 1911
i: 1912
i: 1913
i: 1914
i: 1915
i: 1916
i: 1917
i: 1918
i: 1919
i: 1920
i: 1921
i: 1922
i: 1923
i: 1924
i: 1925
i: 1926
i: 1927
i: 1928
i: 1929
i: 1930
i: 1931
i: 1932
i: 1933
i: 1934
i: 1935
i: 1936
i: 1937
i: 1938
i: 1939
i: 1940
i: 1941
i: 1942
i: 1943
i: 1944
i: 1945
i: 1946
i: 1947
i: 1948
i: 1949
i: 1950
i: 1951
i: 1952
i: 1953
i: 1954
i: 1955
i: 1956
i: 1957
i: 1958
i: 1959
i: 1960
i: 1961
i: 1962
i: 1963


...

i: 886
i: 887
i: 888
i: 889
i: 890
i: 891
i: 892
i: 893
i: 894
i: 895
i: 896
i: 897
i: 898
i: 899
i: 900
i: 901
i: 902
i: 903
i: 904
i: 905
i: 906
i: 907
i: 908
i: 909
i: 910
i: 911
i: 912
i: 913
i: 914
i: 915
i: 916
i: 917
i: 918
i: 919
i: 920
i: 921
i: 922
i: 923
i: 924
i: 925
i: 926
i: 927
i: 928
i: 929
i: 930
i: 931
i: 932
i: 933
i: 934
i: 935
i: 936
i: 937
i: 938
i: 939
i: 940
i: 941
i: 942
i: 943
i: 944
i: 945
i: 946
i: 947
i: 948
i: 949
i: 950
i: 951
i: 952
i: 953
i: 954
i: 955
i: 956
i: 957
i: 958
i: 959
i: 960
i: 961
i: 962
i: 963
i: 964
i: 965
i: 966
i: 967
i: 968
i: 969
i: 970
i: 971
i: 972
i: 973
i: 974
i: 975
i: 976
i: 977
i: 978
i: 979
i: 980
i: 981
i: 982
i: 983
i: 984
i: 985
i: 986
i: 987
i: 988
i: 989
i: 990
i: 991
i: 992
i: 993
i: 994
i: 995
i: 996
i: 997
i: 998
i: 999
i: 1000
i: 1001
i: 1002
i: 1003
i: 1004
i: 1005
i: 1006
i: 1007
i: 1008
i: 1009
i: 1010
i: 1011
i: 1012
i: 1013
i: 1014
i: 1015
i: 1016
i: 1017
i: 1018
i: 1019
i: 1020
i: 1021
i: 1022
i: 1023
i: 1024
i:

i: 2006
i: 2007
i: 2008
i: 2009
i: 2010
i: 2011
i: 2012
i: 2013
i: 2014
i: 2015
i: 2016
i: 2017
i: 2018
i: 2019
i: 2020
i: 2021
i: 2022
i: 2023
i: 2024
i: 2025
i: 2026
i: 2027
i: 2028
i: 2029
i: 2030
i: 2031
i: 2032
i: 2033
i: 2034
i: 2035
i: 2036
i: 2037
i: 2038
i: 2039
i: 2040
i: 2041
i: 2042
i: 2043
i: 2044
i: 2045
i: 2046
i: 2047
i: 2048
i: 2049
i: 2050
i: 2051
i: 2052
i: 2053
i: 2054
i: 2055
i: 2056
i: 2057
i: 2058
i: 2059
i: 2060
i: 2061
i: 2062
i: 2063
i: 2064
i: 2065
i: 2066
i: 2067
i: 2068
i: 2069
i: 2070
i: 2071
i: 2072
i: 2073
i: 2074
i: 2075
i: 2076
i: 2077
i: 2078
i: 2079
i: 2080
i: 2081
i: 2082
i: 2083
i: 2084
i: 2085
i: 2086
i: 2087
i: 2088
i: 2089
i: 2090
i: 2091
i: 2092
i: 2093
i: 2094
i: 2095
i: 2096
i: 2097
i: 2098
i: 2099
i: 2100
i: 2101
i: 2102
i: 2103
i: 2104
i: 2105
i: 2106
i: 2107
i: 2108
i: 2109
i: 2110
i: 2111
i: 2112
i: 2113
i: 2114
i: 2115
i: 2116
i: 2117
i: 2118
i: 2119
i: 2120
i: 2121
i: 2122
i: 2123
i: 2124
i: 2125
i: 2126
i: 2127
i: 2128
i: 2129
i: 2130


i: 368
i: 369
i: 370
i: 371
i: 372
i: 373
i: 374
i: 375
i: 376
i: 377
i: 378
i: 379
i: 380
i: 381
i: 382
i: 383
i: 384
i: 385
i: 386
i: 387
i: 388
i: 389
i: 390
i: 391
i: 392
i: 393
i: 394
i: 395
i: 396
i: 397
i: 398
i: 399
i: 400
i: 401
i: 402
i: 403
i: 404
i: 405
i: 406
i: 407
i: 408
i: 409
i: 410
i: 411
i: 412
i: 413
i: 414
i: 415
i: 416
i: 417
i: 418
i: 419
i: 420
i: 421
i: 422
i: 423
i: 424
i: 425
i: 426
i: 427
i: 428
i: 429
i: 430
i: 431
i: 432
i: 433
i: 434
i: 435
i: 436
i: 437
i: 438
i: 439
i: 440
i: 441
i: 442
i: 443
i: 444
i: 445
i: 446
i: 447
i: 448
i: 449
i: 450
i: 451
i: 452
i: 453
i: 454
i: 455
i: 456
i: 457
i: 458
i: 459
i: 460
i: 461
i: 462
i: 463
i: 464
i: 465
i: 466
i: 467
i: 468
i: 469
i: 470
i: 471
i: 472
i: 473
i: 474
i: 475
i: 476
i: 477
i: 478
i: 479
i: 480
i: 481
i: 482
i: 483
i: 484
i: 485
i: 486
i: 487
i: 488
i: 489
i: 490
i: 491
i: 492
i: 493
i: 494
i: 495
i: 496
i: 497
i: 498
i: 499
i: 500
i: 501
i: 502
i: 503
i: 504
i: 505
i: 506
i: 507
i: 508
i: 509
i: 510

i: 1563
i: 1564
i: 1565
i: 1566
i: 1567
i: 1568
i: 1569
i: 1570
i: 1571
i: 1572
i: 1573
i: 1574
i: 1575
i: 1576
i: 1577
i: 1578
i: 1579
i: 1580
i: 1581
i: 1582
i: 1583
i: 1584
i: 1585
i: 1586
i: 1587
i: 1588
i: 1589
i: 1590
i: 1591
i: 1592
i: 1593
i: 1594
i: 1595
i: 1596
i: 1597
i: 1598
i: 1599
i: 1600
i: 1601
i: 1602
i: 1603
i: 1604
i: 1605
i: 1606
i: 1607
i: 1608
i: 1609
i: 1610
i: 1611
i: 1612
i: 1613
i: 1614
i: 1615
i: 1616
i: 1617
i: 1618
i: 1619
i: 1620
i: 1621
i: 1622
i: 1623
i: 1624
i: 1625
i: 1626
i: 1627
i: 1628
i: 1629
i: 1630
i: 1631
i: 1632
i: 1633
i: 1634
i: 1635
i: 1636
i: 1637
i: 1638
i: 1639
i: 1640
i: 1641
i: 1642
i: 1643
i: 1644
i: 1645
i: 1646
i: 1647
i: 1648
i: 1649
i: 1650
i: 1651
i: 1652
i: 1653
i: 1654
i: 1655
i: 1656
i: 1657
i: 1658
i: 1659
i: 1660
i: 1661
i: 1662
i: 1663
i: 1664
i: 1665
i: 1666
i: 1667
i: 1668
i: 1669
i: 1670
i: 1671
i: 1672
i: 1673
i: 1674
i: 1675
i: 1676
i: 1677
i: 1678
i: 1679
i: 1680
i: 1681
i: 1682
i: 1683
i: 1684
i: 1685
i: 1686
i: 1687


i: 2648
i: 2649
i: 2650
i: 2651
i: 2652
i: 2653
i: 2654
i: 2655
i: 2656
i: 2657
i: 2658
i: 2659
i: 2660
i: 2661
i: 2662
i: 2663
i: 2664
i: 2665
i: 2666
i: 2667
i: 2668
i: 2669
i: 2670
i: 2671
i: 2672
i: 2673
i: 2674
i: 2675
i: 2676
i: 2677
i: 2678
i: 2679
i: 2680
i: 2681
i: 2682
i: 2683
i: 2684
i: 2685
i: 2686
i: 2687
i: 2688
i: 2689
i: 2690
i: 2691
i: 2692
i: 2693
i: 2694
i: 2695
i: 2696
i: 2697
i: 2698
i: 2699
i: 2700
i: 2701
i: 2702
i: 2703
i: 2704
i: 2705
i: 2706
i: 2707
i: 2708
i: 2709
i: 2710
i: 2711
i: 2712
i: 2713
i: 2714
i: 2715
i: 2716
i: 2717
i: 2718
i: 2719
i: 2720
i: 2721
i: 2722
i: 2723
i: 2724
i: 2725
i: 2726
i: 2727
i: 2728
i: 2729
i: 2730
i: 2731
i: 2732
i: 2733
i: 2734
i: 2735
i: 2736
i: 2737
i: 2738
i: 2739
i: 2740
i: 2741
i: 2742
i: 2743
i: 2744
i: 2745
i: 2746
i: 2747
i: 2748
i: 2749
done


In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import math

tf.reset_default_graph()


learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
batch_size = 128
n_epochs = 100

# Import MNIST data
mnist = input_data.read_data_sets('.', one_hot=True)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])


weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={features: batch_features, labels: batch_labels})

        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))



Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Epoch 0   - Validation Accuracy: 0.09799999743700027
Epoch 10  - Validation Accuracy: 0.2248000055551529
Epoch 20  - Validation Accuracy: 0.36000001430511475
Epoch 30  - Validation Accuracy: 0.4724000096321106
Epoch 40  - Validation Accuracy: 0.5450000166893005
Epoch 50  - Validation Accuracy: 0.5982000231742859
Epoch 60  - Validation Accuracy: 0.6403999924659729
Epoch 70  - Validation Accuracy: 0.6665999889373779
Epoch 80  - Validation Accuracy: 0.6862000226974487
Epoch 90  - Validation Accuracy: 0.703000009059906


In [38]:
#tutorial softmax regresion https://www.tensorflow.org/get_started/mnist/pros

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.', one_hot=True)

import tensorflow as tf
sess=tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None,784])
y_ = tf.placeholder(tf.float32, shape=[None,10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())
y = tf.matmul(x,W) + b #should be tf.add? this is broadcasting?

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
optimizer= tf.train.GradientDescentOptimizer(0.5)
grads = optimizer.compute_gradients(cross_entropy)
#print(type(grads))
#print(grads) 
apply_grads = optimizer.apply_gradients(grads_and_vars = grads)

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_12' with dtype float
	 [[Node: Placeholder_12 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_12', defined at:
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/zmq/event

In [42]:
#considered a solved problem at 99% redo the above as a DNN
#source: https://www.tensorflow.org/get_started/mnist/pros


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

print("shape:", mnist.test.images.shape)
print("shape:", mnist.test.labels.shape)

import tensorflow as tf


#using relu neurons, init with slightly positive bias

#def weight_variable(shape):
#    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

#def bias_variable(shape):
#    return tf.Variable(tf.constant(0.1,shape=shape))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


#def conv2d(x,W):
#    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')
#def max_pool_2x2(x):
#    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
#conv layer 1
#5x5 patch, 1 input channel, 32 output channels
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#resahpe x to 4d tensor
x_image = tf.reshape(x,[-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#conv layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#FC layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#readout layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
shape: (10000, 784)
shape: (10000, 10)
test accuracy 0.1419


In [41]:
#above add global step
#add tensorboard
#add gradient visualization
#add learning rate decay
#add estimator model style


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True)

learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/example'

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Create a summary to visualize the first layer ReLU activation
    tf.summary.histogram("relu1", layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Create another summary to visualize the second layer ReLU activation
    tf.summary.histogram("relu2", layer_2)
    # Output layer
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

# Encapsulating all ops into scopes, making Tensorboard's Graph
# Visualization more convenient
with tf.name_scope('Model'):
    # Build model
    pred = multilayer_perceptron(x, weights, biases)

with tf.name_scope('Loss'):
    # Softmax Cross entropy (cost function)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Op to calculate every variable gradient
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    # Op to update all variables according to their gradient
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
# Summarize all gradients
#for grad, var in grads:
#    tf.summary.histogram(var.name, grad)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")





Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Summary name Variable:0 is illegal; using Variable_0 instead.
INFO:tensorflow:Summary name Variable_1:0 is illegal; using Variable_1_0 instead.
INFO:tensorflow:Summary name Variable_2:0 is illegal; using Variable_2_0 instead.
INFO:tensorflow:Summary name Variable_3:0 is illegal; using Variable_3_0 instead.
INFO:tensorflow:Summary name Variable_4:0 is illegal; using Variable_4_0 instead.
INFO:tensorflow:Summary name Variable_5:0 is illegal; using Variable_5_0 instead.
INFO:tensorflow:Summary name Variable_6:0 is illegal; using Variable_6_0 instead.
INFO:tensorflow:Summary name Variable_7:0 is illegal; using Variable_7_0 instead.
INFO:tensorflow:Summary name Variable_8:0 is illegal; using Variable_8_0 instead.
INFO:tensorflow:Summary name Variable_9:0 is illegal; using Variable_9_0 ins

INFO:tensorflow:Summary name Variable_96:0 is illegal; using Variable_96_0 instead.
INFO:tensorflow:Summary name Variable_97:0 is illegal; using Variable_97_0 instead.
INFO:tensorflow:Summary name Variable_98:0 is illegal; using Variable_98_0 instead.
INFO:tensorflow:Summary name Variable_99:0 is illegal; using Variable_99_0 instead.
INFO:tensorflow:Summary name Variable_100:0 is illegal; using Variable_100_0 instead.
INFO:tensorflow:Summary name Variable_101:0 is illegal; using Variable_101_0 instead.
INFO:tensorflow:Summary name Variable_102:0 is illegal; using Variable_102_0 instead.
INFO:tensorflow:Summary name Variable_103:0 is illegal; using Variable_103_0 instead.
INFO:tensorflow:Summary name Variable_104:0 is illegal; using Variable_104_0 instead.
INFO:tensorflow:Summary name Variable_105:0 is illegal; using Variable_105_0 instead.
INFO:tensorflow:Summary name Variable_106:0 is illegal; using Variable_106_0 instead.
INFO:tensorflow:Summary name Variable_107:0 is illegal; using 

InvalidArgumentError: You must feed a value for placeholder tensor 'InputData' with dtype float
	 [[Node: InputData = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'InputData', defined at:
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/zmq/eventloop/ioloop.py"

In [ ]:
#above with keras




In [ ]:
#redoing the housing example from hw1
#tf.nn.l2_loss(t, name=None)



